In [21]:
%load_ext autoreload
%autoreload 2

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import classification_report

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger

import numpy as np
import pandas as pd
from DataGenerator import DataGenerator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
X_train = np.load('./datasets/CNN/X_train_no_edge_frames.npy')
X_test = np.load('./datasets/CNN/X_test_no_edge_frames.npy')
y_train = np.load('./datasets/CNN/y_train_no_edge_frames.npy')
y_test = np.load('./datasets/CNN/y_test_no_edge_frames.npy')

In [9]:
def createModel():
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=(3,3), input_shape=(176,240,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3)))
    
    model.add(Conv2D(filters=32, kernel_size=(3,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3)))
    
    model.add(Conv2D(filters=64, kernel_size=(3,3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3)))
    
    model.add(GlobalAveragePooling2D())
    
#     model.add(Dense(64, activation='relu', input_dim=2048))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(500, activation='relu', input_dim=2048))
    model.add(Flatten())
    model.add(Dense(48, activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [10]:
model = createModel()

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 174, 238, 16)      160       
_________________________________________________________________
batch_normalization_3 (Batch (None, 174, 238, 16)      64        
_________________________________________________________________
activation_3 (Activation)    (None, 174, 238, 16)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 58, 79, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 77, 32)        4640      
_________________________________________________________________
batch_normalization_4 (Batch (None, 56, 77, 32)        128       
_________________________________________________________________
activation_4 (Activation)    (None, 56, 77, 32)       

In [28]:
train_gen = DataGenerator(X_train, y_train, 128)
test_gen = DataGenerator(X_test, y_test, 128)

In [29]:
epochs_to_wait_for_improvement = 10
logging_path = './logs'
models_path = './models'
model_name = 'CNN_no_edge_frames'

early_stopping = EarlyStopping(monitor='val_loss', patience=epochs_to_wait_for_improvement)
checkpoint = ModelCheckpoint(f'{models_path}/{model_name}.h5', monitor='val_loss', save_best_only=True, mode='min')
csv_logger = CSVLogger(f'{logging_path}/{model_name}.log')

callbacks = [early_stopping, checkpoint, csv_logger]

In [27]:
model.fit_generator(generator=train_gen,
                    steps_per_epoch=len(train_gen),
                    #                    steps_per_epoch = int(3800 // batch_size),
                    epochs = 1000,
                    verbose = 1,
                    validation_data = test_gen,
                    validation_steps=len(test_gen),
                    callbacks=callbacks
                    #                    validation_steps = int(950 // batch_size)
                   )

Epoch 1/1000
7/7 [==============================] - 1s 83ms/step - loss: 0.6745 - accuracy: 0.9600 - val_loss: 4.3716 - val_accuracy: 0.0000e+00
Epoch 2/1000
7/7 [==============================] - 1s 79ms/step - loss: 0.6573 - accuracy: 0.9600 - val_loss: 4.2574 - val_accuracy: 0.0100
Epoch 3/1000
7/7 [==============================] - 0s 70ms/step - loss: 0.6757 - accuracy: 0.9600 - val_loss: 4.2855 - val_accuracy: 0.0400
Epoch 4/1000
7/7 [==============================] - 1s 76ms/step - loss: 0.6114 - accuracy: 0.9800 - val_loss: 4.4755 - val_accuracy: 0.0000e+00
Epoch 5/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.6161 - accuracy: 0.9600 - val_loss: 4.3673 - val_accuracy: 0.1000
Epoch 6/1000
7/7 [==============================] - 1s 76ms/step - loss: 0.5804 - accuracy: 0.9700 - val_loss: 4.4482 - val_accuracy: 0.0000e+00
Epoch 7/1000
7/7 [==============================] - 1s 75ms/step - loss: 0.5724 - accuracy: 0.9700 - val_loss: 4.4527 - val_accuracy: 0.0600
E